In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier

In [3]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")


In [ ]:
def transformar_Horario():
    pass
